# Modelo Planeamiento Transmisión DC una etapa
El modelo de planeamiento de la transmisión a muy largo plazo de forma <b>DC</b> es un modelo MINLP (Programación no lineal entero mixto)
Tiene como fin obtener la red futura de transmisión.

## Simplificaciones
El modelo considerado realiza bastantes simplificaciones al problema de planeamiento de la transmissión (<i>TNEP por sus siglas en inglés</i>), entre las cuales se tiene:
<ol>
<li>Modelo DC</li>
<li>Expansión de líneas únicamente</li>
<li>No considera contingencias</li>
<li>No considera cortocircuito</li>
<li>No es multietapa</li>
<li>Entre otras</li>
</ol>

En las siguientes secciones se observa el problema de planeamiento considerado


## Modelo MINLP DC
Como se dijo anteriormente, el modelo considera la expansión en líneas de transmisión sujeto a unas restricciones técnicas tales como:
<ul>
<li>Cumplimiento de las ecuaciones de balance nodal</li>
<li>Límite máximo de transferencia de potencia por las líneas</li>
<li>Límite máximo de potencia de los generadores</li>
</ul>

$$ \min \sum_{ij \in \Omega_L} c_{ij} n_{ij}\\
\textit{s.t.}\\
\sum_{ij \in \Omega_L} f_{ij} - \sum_{ji \in \Omega_L} f_{ji} + g_{i} = d_{i} \quad \forall i \in \Omega_b \\
f_{ij} - b_{ij}(\theta_i-\theta_j)(n_{ij}^0+n_{ij}) = 0 \quad \forall ij \in \Omega_L\\
|f_{ij}| \leq (n_{ij}^0+n_{ij})\overline{f_{ij}} \quad \forall ij \in \Omega_L \\
0 \leq n_{ij} \leq \overline{n_{ij}} \quad \forall ij \in \Omega_L \\
\underline{g_i} \leq g_i \leq \overline{g_i} \quad \forall i \in \Omega_b\\
n_{ij} \in \mathbb{N}\\
\Omega_L \subset \Omega_b \times \Omega_b
 $$



La ecuación $ f_{ij} - b_{ij}(\theta_i-\theta_j)(n_{ij}^0+n_{ij}) = 0 $ es una ecuación no lineal, pero esta puede ser linealizada mediante un modelo disyuntivo.<br>
El modelo disyuntivo agrega una nueva variable $y_{ijk}$ por cada corredor adicional que se pueda agregar en el corredor <i>i-j</i><br>
El nuevo modelo es:

$$ \min \sum_{ij \in \Omega_L} c_{ij} \sum_{k \in K_{ij}} y_{ijk} + \alpha \sum_{i \in \Omega_b} r_i\\
\textit{s.t.}\\
\sum_{ij \in \Omega_L} f_{ij} - \sum_{ji \in \Omega_L} f_{ji} + g_{i} +r_{i}= d_{i} \quad \forall i \in \Omega_b \\
f_{ij} - b_{ij}(\theta_i-\theta_j)n_{ij}^0 - \sum_{k \in K_{ij}} f_{ijk} = 0 \quad \forall ij \in \Omega_L\\
|f_{ij}| \leq (n_{ij}^0+\sum_{k \in K_{ij}} y_{ijk})\overline{f_{ij}} \quad \forall ij \in \Omega_L \\
-M\cdot(1-y_{ijk}) \leq f_{ijk} - b_{ij}(\theta_i - \theta_j) \leq M\cdot(1-y_{ijk}) \quad \forall k \in K_{ij}, ij \in \Omega_L\\
-My_{ijk} \leq f_{ijk} \leq My_{ijk} \quad \forall k \in K_{ij}, ij \in \Omega_L\\
\sum_{k \in K_{ij}} y_{ijk} \leq \overline{n_{ij}} \quad \forall ij \in \Omega_L\\
y_{ijk} \in \{0,1\} \quad \forall k \in K_{ij}, ij \in \Omega_L\\
\underline{g_i} \leq g_i \leq \overline{g_i} \quad \forall i \in \Omega_b\\
\Omega_L \subset \Omega_b \times \Omega_b
 $$



## Modelo en Pyomo
Pyomo es un paquete de python, que se usa para formular, resolver (ayuda de solvers) y analizar modelos de optimización <br>
Es un lenguaje de modelado algebráico. De acuerdo a su [descripción](http://www.pyomo.org/about), **Pyomo** tiene la capacidad de modelar problemas de tipo:
<ul>
<li>LP</li>
<li>QP</li>
<li>NLP</li>
<li><p><strong>MILP</strong></p></li>
<li>MIQP</li>
<li>MINLP</li>
<li>SP</li>
<li>GDP</li>
<li>DAE</li>
<li>Bilevel Programming</li>
<li>MPEC</li>
</ul>


## Sistema a resolver
![sistema](Sistema_4bar_fig.jpg)

## Modelo en pyomo
Los parámetros del modelo son los siguientes:<br>
**\# barras**:4<br>
**\# lineas**:5<br>
<i>Parámetros de las barras</i>

| Barra | Dem  | Gmin | Gmax |
| :---- | :--- | :--- | :--- |
| 1     | 0    | 0    | 1,05 |
| 2     | 0,6  | 0    | 0    |
| 3     | 0,2  | 0    | 0    |
| 4     | 0,25 | 0    | 0    |

<i>Parámetros de las líneas</i>

| barra i | barra j | n0  | nmax | X reac | capacidad | costo |
| :------ | :------ | :-- | :--- | :----- | :-------- | :---- |
| 1       | 2       | 0   | 2    | 3      | 0,35      | 3     |
| 1       | 3       | 1   | 2    | 2      | 0,4       | 2     |
| 1       | 4       | 0   | 2    | 2      | 0,4       | 2     |
| 2       | 4       | 0   | 2    | 2      | 0,4       | 2     |
| 2       | 3       | 1   | 3    | 2      | 0,4       | 2     |




## Importar Librerías

In [1]:
import time # contador de tiempo
import pyomo.environ as pyo # importar la libreria pyomo
import pandas as pd # librería para procesamiento dataframe 


## Importar datos del sistema

In [2]:
import system_4bar as raw_system
lineas = pd.DataFrame(raw_system.datos_lineas,columns=['bus_i','bus_j','n0','nmax','X','cap','costo'])
# genera un indice Linea_busi_busj para nombres de las variables tambien
index_lineas = ['Linea_']*raw_system.num_lineas + lineas['bus_i'].astype('string') + ["_"]*raw_system.num_lineas + lineas['bus_j'].astype('string')
lineas.index = index_lineas
barras = pd.DataFrame(raw_system.datos_bus,columns=['bus','dem','gmin','gmax'])
# genera un indice barra_i para nombres de las variables tambien
index_barras = ['barra_']*raw_system.num_barras + barras['bus'].astype('string')
barras.index = index_barras

## Iniciar modelo de pyomo
Existen dos formas de modelo:
* Concrete
* Abstract
En esta guía se usa el modelo concreto

In [3]:
model = pyo.ConcreteModel()

Declarar conjuntos:
* Se puede realizar por medio de diccionarios y de listas externas
* En modelos estructurados se requiere el uso de conjuntos sobre los cuales iteran las variables

Conjunto de líneas y barras

In [4]:
# Conjunto de lineas y de barras
model.set_lineas = pyo.Set(initialize=index_lineas)
model.set_barras = pyo.Set(initialize=index_barras)

Conjunto para modelo disyuntivo <br>
* Recordar que es un conjunto basado en línea x nmax <br>
\{ij\}x\{1,2,3\} = \{(ij,1),(ij,2),(ij,3)\} <br>

In [5]:
# declarar conjunto para variable yijk (variable binaria para proceso de separacion y disyunción)
set_nmax = range(max(lineas['nmax'].astype('int')))
def set_disyuntivo(m):
    ans = [] # arranca vacia la declaracion de parámetros
    for i,linea in lineas.iterrows():
        if(linea['nmax']>0):
            for j in set_nmax:
                ans.append((i,j))
                #print(i,j)
    return ans

def filter_disyuntivo(m,ij,k):
    if k <= (lineas.loc[ij,'nmax']-1):
        return True
    else:
        return False
    

model.set_yijk = pyo.Set(within = model.set_lineas * set_nmax,
                         initialize=set_disyuntivo,filter=filter_disyuntivo)

Declarar un conjunto de generadores <br>
es un subconjunto del conjunto de nodos


In [6]:
# declarar conjunto de generacion (que es un subconjunto de nodos)
def subset_gen(m,i):
    if (barras.loc[i,'gmax']>0):
        return True
    else:
        return False
model.set_gen= pyo.Set(initialize=model.set_barras,within=model.set_barras,filter=subset_gen)

# Declaración de variables
$f_{ij}$: Flujo de potencia en el corredor i-j<br>
$\theta_i$: Ángulo del nodo <i>i</i><br>
$r_i$: Racionamiento del nodo <i>i</i><br>
$y_{ijk}$: variable binaria para habilitar la línea *k* en el corredor *i-j*<br>
$f_{ijk}$: flujo de potencia por la línea *k* en el corredor *i-j*<br>
$g_{i}$: generación en el nodo *i*


In [7]:
# Declaracion de variables
model.fij = pyo.Var(model.set_lineas,name= 'f_ij') # flujo f_ij flujo corredor i-j
model.theta = pyo.Var(model.set_barras,name='theta_i', initialize =0) # angulo de flujo entre las líneas
model.rac = pyo.Var(model.set_barras,name='r_i', initialize = 0,bounds=(0,None)) # variable de racionamiento del sistema (se implementa como variable de holgura)
model.yijk = pyo.Var(model.set_yijk,within=pyo.Binary) # declaracion de las variables disyuntivas
model.fijk = pyo.Var(model.set_yijk,within=pyo.Reals) # variable fijk para separacion de flujo 
# declaracion de límites de las variables
def gen_bounds(model,i):
    return (barras.loc[i,'gmin'],barras.loc[i,'gmax'])
model.gen = pyo.Var(model.set_gen, within=pyo.Reals, bounds=gen_bounds)

# Restricciones


## Balance Nodal
$\sum_{ij \in \Omega_L} f_{ij} - \sum_{ji \in \Omega_L} f_{ji} + g_{i} +r_{i}= d_{i} \quad \forall i \in \Omega_b$


In [8]:
# Modelo de balance de potencia nodal
def balance_nodal(m,i):
    lineas_in = index_lineas[(lineas['bus_i']==barras.loc[i,'bus']).tolist()]
    lineas_out = index_lineas[(lineas['bus_j']==barras.loc[i,'bus']).tolist()]
    if (i in model.set_gen):
        return sum(m.fij[linea] for linea in lineas_in)-sum(m.fij[linea] for linea in lineas_out) \
            + m.gen[i] + m.rac[i] == barras.loc[i,'dem']
    else:
        return sum(m.fij[linea] for linea in lineas_in)-sum(m.fij[linea] for linea in lineas_out) \
            +  m.rac[i] == barras.loc[i,'dem']
model.balance_nodal = pyo.Constraint(model.set_barras,rule= balance_nodal)

## Relación flujo - tensión angular
$f_{ij} - b_{ij}(\theta_i-\theta_j)n_{ij}^0 - \sum_{k \in K_{ij}} f_{ijk} = 0 \quad \forall ij \in \Omega_L$

In [9]:
# modelo diferencia angular flujo
def dif_angular(m,ij):
    # determinar los nodos a los cuales se encuentra conectada la linea
    bus1 = 'barra_'+lineas.loc[ij,'bus_i'].astype('str')
    bus2 = 'barra_'+lineas.loc[ij,'bus_j'].astype('str')
    return m.fij[ij] - 1/lineas.loc[ij,'X']*(m.theta[bus1]-m.theta[bus2])*lineas.loc[ij,'n0']-sum(m.fijk[ij,k] for k in range(lineas.loc[ij,'nmax'].astype('int'))) == 0
model.flujo_theta = pyo.Constraint(model.set_lineas,rule = dif_angular)  

## Límite de capacidad de la línea
$|f_{ij}| \leq (n_{ij}^0+\sum_{k \in K_{ij}} y_{ijk})\overline{f_{ij}} \quad \forall ij \in \Omega_L$<br>
Es equivalente a:<br>
$-F \leq f_{ij} \leq F$

In [10]:
def line_capacity_1(m,linea):
    return -(lineas.loc[linea,'n0']+sum(m.yijk[linea,k] for k in range(lineas.loc[linea,'nmax'].astype('int'))))*lineas.loc[linea,'cap'] <= m.fij[linea]
def line_capacity_2(m,linea):
    return m.fij[linea] <= (lineas.loc[linea,'n0']+sum(m.yijk[linea,k] for k in range(lineas.loc[linea,'nmax'].astype('int'))))*lineas.loc[linea,'cap']
model.limit_line1 = pyo.Constraint(model.set_lineas,rule=line_capacity_1)
model.limit_line2 = pyo.Constraint(model.set_lineas,rule=line_capacity_2)

## Modelo restricción disyuntiva
$$-M\cdot(1-y_{ijk}) \leq f_{ijk} - b_{ij}(\theta_i - \theta_j) \leq M\cdot(1-y_{ijk}) \quad \forall k \in K_{ij}, ij \in \Omega_L$$

In [11]:
model.M=1e3 # parámetro big-M
def rest_disyuntiva_1_1(m,ij,k):
    # determinar los nodos a los cuales se encuentra conectada la linea
    bus1 = 'barra_'+lineas.loc[ij,'bus_i'].astype('str')
    bus2 = 'barra_'+lineas.loc[ij,'bus_j'].astype('str')
    # retornar la ecuacion
    if (k<=lineas.loc[ij,'nmax'].astype('int')):
        return -m.M*(1-m.yijk[ij,k]) <= m.fijk[ij,k] - 1/lineas.loc[ij,'X']*(m.theta[bus1]-m.theta[bus2])
    else:
        return pyo.Constraint.Skip
def rest_disyuntiva_1_2(m,ij,k):
    # determinar los nodos a los cuales se encuentra conectada la linea
    bus1 = 'barra_'+lineas.loc[ij,'bus_i'].astype('str')
    bus2 = 'barra_'+lineas.loc[ij,'bus_j'].astype('str')
    # retornar la ecuacion
    if (k<=lineas.loc[ij,'nmax'].astype('int')):
        return m.fijk[ij,k] - 1/lineas.loc[ij,'X']*(m.theta[bus1]-m.theta[bus2]) <= m.M*(1-m.yijk[ij,k]) 
    else:
        return pyo.COnstraint.Skip
model.disjoint1_1 = pyo.Constraint(model.set_yijk,rule = rest_disyuntiva_1_1)
model.disjoint1_2 = pyo.Constraint(model.set_yijk,rule = rest_disyuntiva_1_2)

$$-My_{ijk} \leq f_{ijk} \leq My_{ijk} \quad \forall k \in K_{ij}, ij \in \Omega_L$$


In [12]:
def rest_disyuntiva_2_1(m,ij,k):
    if (k<=lineas.loc[ij,'nmax'].astype('int')):
        return -1*m.M*m.yijk[ij,k] <= m.fijk[ij,k]
    else:
        return pyo.Constraint.Skip
def rest_disyuntiva_2_2(m,ij,k):
    if (k<=lineas.loc[ij,'nmax'].astype('int')):
        return  m.fijk[ij,k] <= m.M*m.yijk[ij,k]
    else:
        return Constraint.Skip

model.disjoint2_1 = pyo.Constraint(model.set_yijk,rule=rest_disyuntiva_2_1)
model.disjoint2_2 = pyo.Constraint(model.set_yijk,rule=rest_disyuntiva_2_2)

## Función objetivo
$$\min \sum_{ij \in \Omega_L} c_{ij} \sum_{k \in K_{ij}} y_{ijk} + \alpha \sum_{i \in \Omega_b} r_i$$

In [13]:
# funcion objetivo
def FO(m):
    return 10000*sum(m.rac[i] for i in m.set_barras) + sum(lineas.loc[ij,'costo']*sum(m.yijk[ij,k] for k in range(lineas.loc[ij,'nmax'].astype('int'))) for ij in m.set_lineas)
# incluir en el modelo
model.objetivo = pyo.Objective(rule=FO,sense = pyo.minimize)

*Impresión del modelo*

In [14]:
model.pprint()

6 Set Declarations
    set_barras : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'barra_1', 'barra_2', 'barra_3', 'barra_4'}
    set_gen : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain     : Size : Members
        None :     1 : set_barras :    1 : {'barra_1',}
    set_lineas : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'Linea_1_2', 'Linea_1_3', 'Linea_1_4', 'Linea_2_4', 'Linea_2_3'}
    set_yijk : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain          : Size : Members
        None :     2 : set_yijk_domain :   11 : {('Linea_1_2', 0), ('Linea_1_2', 1), ('Linea_1_3', 0), ('Linea_1_3', 1), ('Linea_1_4', 0), ('Linea_1_4', 1), ('Linea_2_4', 0), ('Linea_2_4', 1), ('Linea_2_3', 0), ('Linea_2_3', 1), ('Linea_2_3', 2)}
    set_yijk_domain : Size=1, Index=None, Ordered=False
        Key  : Di

## Solución del Modelo
**Pyomo** solamente realiza el modelo, para su solución se debe invocar un solver. Como el problema es MILP se usará el solver gratuito [glpk](https://www.gnu.org/software/glpk/)

In [15]:
solver = pyo.SolverFactory('glpk') # invocación del solver
# resolver el modelo
solver.solve(model, tee = True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\CAMILO~1\AppData\Local\Temp\tmpubk5_l0o.glpk.raw --wglp
 C:\Users\CAMILO~1\AppData\Local\Temp\tmp0dlcubrv.glpk.glp --cpxlp C:\Users\CAMILO~1\AppData\Local\Temp\tmpkjdbwlrf.pyomo.lp
Reading problem data from 'C:\Users\CAMILO~1\AppData\Local\Temp\tmpkjdbwlrf.pyomo.lp'...
C:\Users\CAMILO~1\AppData\Local\Temp\tmpkjdbwlrf.pyomo.lp:452: warning: lower bound of variable 'x14' redefined
C:\Users\CAMILO~1\AppData\Local\Temp\tmpkjdbwlrf.pyomo.lp:452: warning: upper bound of variable 'x14' redefined
64 rows, 37 columns, 200 non-zeros
11 integer variables, all of which are binary
463 lines were read
Writing problem data to 'C:\Users\CAMILO~1\AppData\Local\Temp\tmp0dlcubrv.glpk.glp'...
403 lines were written
GLPK Integer Optimizer, v4.65
64 rows, 37 columns, 200 non-zeros
11 integer variables, all of which are binary
Preprocessing...
63 rows, 32 columns, 195 non-zeros
11 integer variables, all of which a

{'Problem': [{'Name': 'unknown', 'Lower bound': 5.99999999999889, 'Upper bound': 5.99999999999889, 'Number of objectives': 1, 'Number of constraints': 64, 'Number of variables': 37, 'Number of nonzeros': 200, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '27', 'Number of created subproblems': '27'}}, 'Error rc': 0, 'Time': 0.11655306816101074}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

## Imprimir Resultados

In [16]:
print('#################FUNCION OBJETIVO####################')
print(pyo.value(model.objetivo))
# corredores nuevos usados
for ij in model.set_lineas:
    x = pyo.value(sum(model.yijk[ij,k]for k in range(lineas.loc[ij,'nmax'].astype('int'))))
    print("Corredores nuevos en "+ij+": "+str(x))

#################FUNCION OBJETIVO####################
5.99999999999889
Corredores nuevos en Linea_1_2: 0.0
Corredores nuevos en Linea_1_3: 1.0
Corredores nuevos en Linea_1_4: 1.0
Corredores nuevos en Linea_2_4: 0.0
Corredores nuevos en Linea_2_3: 1.0


Realizar Pruebas Adicionales para revisar otras condiciones de solución